# Bash function

In [1]:
def bash(string,name,queue):
  script = "%s.sh"%name
  with open(script, "w") as text_file:
    text_file.write(string)
  !chmod +x $script
  !qsub -q $queue $script
  !rm $script
  
ws = '/beegfs/work/workspace/ws/ho_rettu74-genomes-0/jupyterlab'

#Create list of file indexes to access it later
import os
!rm -r Raw_data/.ipynb_checkpoints
reads = sorted(list(set([f.split('_')[0].split('.')[-1] for f in os.listdir('Raw_data')])))
done = sorted(list(set([f for f in os.listdir('Spades_assembly')])))
print('Raw reads:', reads)
print('Processed:', done)
print('Assemblies are missing for (if empty all good!): ', [r for r in reads if r not in done])

rm: cannot remove ‘Raw_data/.ipynb_checkpoints’: No such file or directory


FileNotFoundError: [Errno 2] No such file or directory: 'Raw_data'

# Unzip and move files

In [19]:
!unzip X204*

Archive:  X204SC21100034-Z01-F003.zip
   creating: X204SC21100034-Z01-F003/
 extracting: X204SC21100034-Z01-F003/MD5.zip  
  inflating: X204SC21100034-Z01-F003/MD5.txt  
  inflating: X204SC21100034-Z01-F003/checkSize.xls  
   creating: X204SC21100034-Z01-F003/report/
 extracting: X204SC21100034-Z01-F003/report/X204SC21100034-Z01-F003_final_20220315070504.zip  
   creating: X204SC21100034-Z01-F003/raw_data/
   creating: X204SC21100034-Z01-F003/raw_data/LTX114t4_1/
  inflating: X204SC21100034-Z01-F003/raw_data/LTX114t4_1/LTX114t4_1_FSFP220075839-1r_HWWW2DSX2_L4_1.fq.gz  
  inflating: X204SC21100034-Z01-F003/raw_data/LTX114t4_1/MD5.txt  
  inflating: X204SC21100034-Z01-F003/raw_data/LTX114t4_1/LTX114t4_1_FDSW220075839-1r_HWYCLDSX2_L3_1.fq.gz  
  inflating: X204SC21100034-Z01-F003/raw_data/LTX114t4_1/LTX114t4_1_FSFP220075839-1r_HWWW2DSX2_L4_2.fq.gz  
  inflating: X204SC21100034-Z01-F003/raw_data/LTX114t4_1/LTX114t4_1_FDSW220075839-1r_HWYCLDSX2_L3_2.fq.gz  
   creating: X204SC21100034-Z01-F

In [22]:
# Check if all samples sequenced in 1 line or not already sequenced earlier
for dir in os.listdir('X204SC21100034-Z01-F003/raw_data/'):
  if dir in os.listdir('Raw_data'):
    print('Ups! Already sequenced, concatenate!',dir)
  if dir != 'Rawdata_Readme.pdf' and len(os.listdir('X204SC21100034-Z01-F003/raw_data/'+dir)) > 3:
    print('Ups! Sequenced in 2 lines, concatenate!',dir)

Ups! Sequenced in 2 lines, concatenate! J22
Ups! Sequenced in 2 lines, concatenate! I26


In [23]:
# move files to Raw_reads
for dir in os.listdir('X204SC21100034-Z01-F003/raw_data/'):
  if dir != 'Rawdata_Readme.pdf':
    !mv X204SC21100034-Z01-F003/raw_data/$dir/*fq.gz Raw_data/

# Concatenate same samples

In [27]:
reads = [f.split('_')[0].split('.')[-1] for f in os.listdir(ws+'/Raw_data')]
reads = sorted(list(set(reads)))
for read in reads:
  if [f.split('_')[0].split('.')[-1] for f in os.listdir(ws+'/Raw_data')].count(read) > 3:
    inp1 = ws+'/Raw_data/%s_*_1.fq.gz'% read
    out1 = ws+'/Raw_data/%s_1.fq.gz'  % read
    inp2 = ws+'/Raw_data/%s_*_2.fq.gz'% read
    out2 = ws+'/Raw_data/%s_2.fq.gz'  % read
    !cat $inp1 > $out1
    !cat $inp2 > $out2 
    !rm $inp1 $inp2

# Simplify read_names

In [28]:
reads = [f.split('_')[0].split('.')[-1] for f in os.listdir(ws+'/Raw_data')]
reads = sorted(list(set(reads)))
rr = ws+'/Raw_data'
for read in reads:
  inp1 = rr+'/%s_*_1.fq.gz' %read
  out1 = rr+'/%s_1.fq.gz' %read
  inp2 = rr+'/%s_*_2.fq.gz' %read
  out2 = rr+'/%s_2.fq.gz' %read
  !mv $inp1 $out1
  !mv $inp2 $out2

mv: cannot stat ‘/beegfs/work/workspace/ws/ho_rettu74-genomes-0/jupyterlab/Raw_data/I26_*_1.fq.gz’: No such file or directory
mv: cannot stat ‘/beegfs/work/workspace/ws/ho_rettu74-genomes-0/jupyterlab/Raw_data/I26_*_2.fq.gz’: No such file or directory
mv: cannot stat ‘/beegfs/work/workspace/ws/ho_rettu74-genomes-0/jupyterlab/Raw_data/I27_*_1.fq.gz’: No such file or directory
mv: cannot stat ‘/beegfs/work/workspace/ws/ho_rettu74-genomes-0/jupyterlab/Raw_data/I27_*_2.fq.gz’: No such file or directory
mv: cannot stat ‘/beegfs/work/workspace/ws/ho_rettu74-genomes-0/jupyterlab/Raw_data/J22_*_1.fq.gz’: No such file or directory
mv: cannot stat ‘/beegfs/work/workspace/ws/ho_rettu74-genomes-0/jupyterlab/Raw_data/J22_*_2.fq.gz’: No such file or directory
mv: cannot stat ‘/beegfs/work/workspace/ws/ho_rettu74-genomes-0/jupyterlab/Raw_data/J39_*_1.fq.gz’: No such file or directory
mv: cannot stat ‘/beegfs/work/workspace/ws/ho_rettu74-genomes-0/jupyterlab/Raw_data/J39_*_2.fq.gz’: No such file or di

# BBtools

In [29]:
# remove sequencing artifacts, adapters, merge reads and QC
string = '''
#PBS -l nodes=1:ppn=12
#PBS -l walltime=01:00:00
#PBS -l mem=60gb
#PBS -S /bin/bash
source /home/ho/ho_genetics/ho_rettu74/miniconda3/etc/profile.d/conda.sh
conda activate genomes
bbduk.sh in1=%s in2=%s out1=%s out2=%s outm=%s k=31 ftm=5 ref=%s t=12
bbduk.sh in1=%s in2=%s out1=%s out2=%s ref=%s ktrim=r k=23 mink=11 hdist=1 tbo=t qtrim=r trimq=20 t=12
bbmerge-auto.sh in1=%s in2=%s out=%s outu1=%s outu2=%s rem k=62 extend2=50 ecct t=12 -Xmx60g
bbduk.sh in=%s out=%s qtrim=r trimq=10 minlen=50 t=12
bbduk.sh in1=%s in2=%s out1=%s out2=%s qtrim=r trimq=10 minlen=50 t=12
conda deactivate
'''
nA = ws+'/NoArt_reads'
na = ws+'/NoAdapt_reads'
me = ws+'/Merged_reads'
QC = ws+'/QCreads'
rr = ws+'/Raw_data'
!mkdir $nA $na $me $QC
import os
for read in reads:
  if read in done: continue
  # if read != "B003" : continue
  in1  = rr+'/%s_1.fq.gz'           % read
  in2  = rr+'/%s_2.fq.gz'           % read
  om   = nA+'/%s_matched.fq.gz'     % read
  o1   = nA+'/%s_unmatched_1.fq.gz' % read
  o2   = nA+'/%s_unmatched_2.fq.gz' % read
  o3   = na+'/%s_trimmed_1.fq.gz'   % read
  o4   = na+'/%s_trimmed_2.fq.gz'   % read
  mer  = me+'/%s_merged.fq.gz'      % read
  unm1 = me+'/%s_unmerged_1.fq.gz'  % read
  unm2 = me+'/%s_unmerged_2.fq.gz'  % read
  mQC  = QC+'/%s_merged.fq.gz'      % read
  uQC1 = QC+'/%s_unmerged_1.fq.gz'  % read
  uQC2 = QC+'/%s_unmerged_2.fq.gz'% read
  r1  = '/home/ho/ho_genetics/ho_rettu74/miniconda3/envs/genomes/bbtools/lib/resources/sequencing_artifacts.fa.gz'
  r2  = '/home/ho/ho_genetics/ho_rettu74/miniconda3/envs/genomes/bbtools/lib/resources/adapters.fa'
  if os.path.exists(uQC1): continue
  bash(string%(in1,in2,o1,o2,om,r1,o1,o2,o3,o4,r2,o3,o4,mer,unm1,unm2,mer,mQC,unm1,unm2,uQC1,uQC2),
       'BBTools_%s'%read,'short')

10490117
10490118
10490119
10490120


In [30]:
!mkdir Stats
!mv *.sh.e* Stats/

mv: cannot stat ‘*.sh.e*’: No such file or directory


# Assembly with SPAdes

In [5]:
# Assembly with spades and Quast
string = '''
#PBS -l nodes=1:ppn=12
#PBS -l walltime=2:00:00
#PBS -l mem=60gb
#PBS -S /bin/bash
source /home/ho/ho_genetics/ho_rettu74/miniconda3/etc/profile.d/conda.sh
conda activate genomes
spades.py -k 21,33,55,77 -1 %s -2 %s --merged %s -o %s -m 80 -t 12 --isolate
quast -t 12 -o %s %s
conda deactivate
'''
QC = ws+'/QCreads'
Sp = ws+'/Spades_assembly'
!mkdir $Sp
for read in reads:
  if read in done: continue
  # if read != "B003" : continue
  mer  = QC+'/%s_merged.fq.gz' % read
  unm1 = QC+'/%s_unmerged_1.fq.gz'% read
  unm2 = QC+'/%s_unmerged_2.fq.gz'% read
  out  = Sp+'/'+read
  qua  = Sp+'/'+read+'/Quast'
  ass  = Sp+'/%s/contigs.fasta'%read
  if os.path.exists(out): continue
  bash(string%(unm1,unm2,mer,out,qua,ass),'%s_Spades'%read,'short')

mkdir: cannot create directory ‘/beegfs/work/workspace/ws/ho_rettu74-genomes-0/jupyterlab/Spades_assembly’: File exists
10490277
10490278
10490279
10490280


In [3]:
# move contigs to the new directory with a report
!mkdir contigs
for read in reads:
  oldass = 'Spades_assembly/%s/contigs.fasta'%read
  newass = 'contigs/%s_contigs.fasta'%read
  oldqua = 'Spades_assembly/%s/Quast/report.pdf'%read
  newqua = 'contigs/%s_report.pdf'%read
  !cp $oldass $newass
  !cp $oldqua $newqua

mkdir: cannot create directory ‘contigs’: File exists


In [7]:
# zip contigs folder to download
!zip -r Trycycler.zip Trycycler

updating: Trycycler/ (stored 0%)
updating: Trycycler/assemblies/ (stored 0%)
updating: Trycycler/assemblies/B98/ (stored 0%)
updating: Trycycler/assemblies/B98/assembly_06/ (stored 0%)
updating: Trycycler/assemblies/B98/assembly_06/20-repeat/ (stored 0%)


In [19]:
# clean Spades directory to save some space
keep = ['Quast','contigs.fasta','contigs.paths','scaffolds.fasta','scaffolds.paths','spades.log']
for read in reads:
  for file in os.listdir('Spades_assembly/%s'%read):
    if file not in keep:
      path = 'Spades_assembly/%s/%s'%(read,file)
      !rm -rf $path

In [21]:
##PROKKA
# Annotation with Prokka
string = '''
#PBS -l nodes=1:ppn=2
#PBS -l walltime=1:00:00
#PBS -l mem=24gb
#PBS -S /bin/bash
source /home/ho/ho_genetics/ho_rettu74/miniconda3/etc/profile.d/conda.sh
conda activate prokka
prokka %s --addgenes --outdir %s
conda deactivate
'''
contigs = ws+'/contigs'
prokka = ws+'/prokka'
!mkdir $prokka
for read in reads:
  if read == "B002": continue
  contig = contigs+ "/%s_contigs.fasta" % read
  outdir = ws+"/prokka/%s" % read
  bash(string%(contig,outdir),'%s_prokka'%read,'short')

10490282
10490283
10490284
10490285


In [5]:
# zip prokka folder to download
!zip -r prokka.zip prokka

  adding: prokka/ (stored 0%)
  adding: prokka/J22/ (stored 0%)
  adding: prokka/J22/PROKKA_04062022.gbk (deflated 69%)
  adding: prokka/J22/PROKKA_04062022.sqn (deflated 84%)
  adding: prokka/J22/PROKKA_04062022.tbl (deflated 84%)
  adding: prokka/J22/PROKKA_04062022.log (deflated 79%)
  adding: prokka/J22/PROKKA_04062022.faa (deflated 43%)
  adding: prokka/J22/PROKKA_04062022.fsa (deflated 69%)
  adding: prokka/J22/PROKKA_04062022.txt (deflated 15%)
  adding: prokka/J22/PROKKA_04062022.fna (deflated 69%)
  adding: prokka/J22/PROKKA_04062022.tsv (deflated 75%)
  adding: prokka/J22/PROKKA_04062022.ffn (deflated 69%)
  adding: prokka/J22/PROKKA_04062022.err (deflated 96%)
  adding: prokka/J22/PROKKA_04062022.gff (deflated 74%)
  adding: prokka/J39/ (stored 0%)
  adding: prokka/J39/PROKKA_04062022.gbk (deflated 73%)
  adding: prokka/J39/PROKKA_04062022.sqn (deflated 86%)
  adding: prokka/J39/PROKKA_04062022.tbl (deflated 84%)
  adding: prokka/J39/PROKKA_04062022.log (deflated 82%)
  addi